In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, ManagedOnlineDeployment, ManagedOnlineEndpoint
from azure.identity import DefaultAzureCredential
import datetime

# Connect to Azure ML workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="19209592-5fb7-44f5-b480-296bb2226284",
    resource_group_name="ml-rg",
    workspace_name="ml-workspace"
)


In [2]:
# Generate unique endpoint name using timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")
endpoint_name = f"car-price-endpoint-{timestamp}"
print("✅ Endpoint name:", endpoint_name)


✅ Endpoint name: car-price-endpoint-202508080328


In [3]:
env = Environment(
    name="dp100-inference-env",
    description="Environment for car price prediction model scoring",
    conda_file="dp100_eda_env.yml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest"
)

env = ml_client.environments.create_or_update(env)
print("✅ Environment registered:", env.name)


✅ Environment registered: dp100-inference-env


In [ ]:
# ✅ STEP 1: Import Libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment, ManagedOnlineDeployment, ManagedOnlineEndpoint
import datetime

# ✅ STEP 2: Connect to Azure ML Workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="19209592-5fb7-44f5-b480-296bb2226284",
    resource_group_name="ml-rg",
    workspace_name="ml-workspace"
)

# ✅ STEP 3: Create Unique Endpoint Name
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")
endpoint_name = f"car-price-endpoint-{timestamp}"
print("✅ Endpoint name:", endpoint_name)

# ✅ STEP 4: Create the Azure ML Environment
env = Environment(
    name="dp100-inference-env",
    description="Environment for car price prediction model scoring",
    conda_file="dp100_eda_env.yml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest"
)

env = ml_client.environments.create_or_update(env)
print("✅ Environment registered:", env.name)

# ✅ STEP 5: Create the Managed Online Endpoint (for ACI)
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Managed endpoint for car price model",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("✅ Endpoint created:", endpoint.name)

# ✅ STEP 6: Deploy the Model to ACI
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model="car_price_model:3",  # Update version if needed
    environment=env,
    code_configuration={
        "code": "./",  # assumes score.py is in current folder
        "scoring_script": "score.py"
    },
    instance_type="Standard_DS2_v2",
    instance_count=1
)

ml_client.online_deployments.begin_create_or_update(deployment).result()
print("✅ Deployment created: blue")

# ✅ STEP 7: Set 'blue' as default deployment
ml_client.online_endpoints.begin_update(
    name=endpoint_name,
    defaults={"deployment_name": "blue"}
).result()
print("✅ Deployment set as default for endpoint")

# ✅ STEP 8: Print Endpoint URI and Access Key (for later use)
endpoint = ml_client.online_endpoints.get(endpoint_name)
print("🔗 Scoring URI:", endpoint.scoring_uri)

keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
print("🔐 Primary Key:", keys.primary_key)
